In [2]:
import os
import glob

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from typing import *
from collections import Counter

from sklearn import model_selection

In [3]:
# Note: this is a slightly modified version of the code from the preprocessing_pipeline.py 
# script that can stand alone in a notebook

# to skip scaling... 

def get_text_data(input_dir):
    """
    Purpose: preprocess liwc and nrc
    Input
        input_dir {string} : path to input_directory (ex, "~/Train")
    Output:
        id_list {numpy array of strings}: array of user ids sorted alphabetically,
                                        to determine order of features and labels DataFrames
        text_data {pandas DataFrame of float}: unscaled text data (liwc and nrc combined)
    """
    # Load and sort text data
    liwc = pd.read_csv(os.path.join(input_dir, 'Text/liwc.csv'), sep = ',')
    liwc = liwc.sort_values(by=['userId'])

    nrc = pd.read_csv(os.path.join(input_dir, 'Text/nrc.csv'), sep = ',')
    nrc = nrc.sort_values(by=['userId'])

    # Build list of subject ids ordered alphabetically
    # Check if same subject lists in both sorted DataFrames (liwc and nrc)
    if np.array_equal(liwc['userId'], nrc['userId']):
        id_list = liwc['userId'].to_numpy()
    else:
        raise Exception('userIds do not match between liwc and nrc data')

    # merge liwc and nrc DataFrames using userId as index
    liwc.set_index('userId', inplace=True)
    nrc.set_index('userId', inplace=True)

    text_data = pd.concat([liwc, nrc], axis=1, sort=False)

    return id_list, text_data


def get_image_clean(sub_ids, oxford, means):
    '''
    Purpose: preprocess oxford metrics derived from profile pictures (part 2)
    Input:
        sub_ids {numpy array of strings}: ordered list of userIDs
        oxford {pandas DataFrame of floats}: unscaled oxford features of users with 1+ face
        means {list of float}: mean values for each feature averaged from train set,
                    to replace missing values for userids with no face (train and test set)
    Output:
        image_data {pandas DataFrame of float}: unscaled oxford image data
                with mean values replacing missing entries
    '''
    # list of ids with at least one face on image: 7174 out of 9500 in train set
    ox_list = np.sort(oxford['userId'].unique(), axis=None)
    # list of ids in text_list who have no face metrics in oxford.csv (2326 in train set)
    ox_noface = np.setdiff1d(sub_ids, ox_list)

    # Create DataFrame for userids with no face (1 row per userid)
    # values are mean metrics averaged from users with entries (training set)
    ox_nf = pd.DataFrame(ox_noface, columns = ['userId'])
    columns = oxford.columns[2:].tolist()
    for column, mean in zip(columns, means):
        ox_nf.insert(loc=ox_nf.shape[1], column=column, value=mean, allow_duplicates=True)
    # insert column 'noface' = 1 if no face in image, else 0
    ox_nf.insert(loc=ox_nf.shape[1], column='noface', value=1, allow_duplicates=True)
    # insert column 'multiface' = 1 if many faces in image, else 0
    ox_nf.insert(loc=ox_nf.shape[1], column='multiface', value=0, allow_duplicates=True)
    ox_nf.set_index('userId', inplace=True)

    # Format DataFrame from userids with 1+ face
    # insert column 'noface' = 1 if no face in image, else 0
    oxford.insert(loc=oxford.shape[1], column='noface', value=0, allow_duplicates=True)
    # list userIds with multiple faces (714 in train set)
    ox_multiples = oxford['userId'][oxford['userId'].duplicated()].tolist()
    # insert column 'multiface' = 1 if many faces in image, else 0
    oxford.insert(loc=oxford.shape[1], column='multiface', value=0, allow_duplicates=True)
    multi_mask = pd.Series([uid in ox_multiples for uid in oxford['userId']])
    i = oxford[multi_mask].index
    oxford.loc[i, 'multiface'] = 1
    # drop duplicate entries with same userId (keep first entry per userId)
    oxford.drop_duplicates(subset ='userId', keep='first', inplace=True)

    # merge the two DataFrames
    oxford.drop(['faceID'], axis=1, inplace=True)
    oxford.set_index('userId', inplace=True)
    image_data = pd.concat([ox_nf, oxford], axis=0, sort=False).sort_values(by=['userId'])

    if not np.array_equal(image_data.index, sub_ids):
        raise Exception('userIds do not match between oxford file and id list')

        ### ADD THE HIGH LEVEL PRE-PROCESSING WHEN DONE (22 NOV)
        
    return image_data


def get_image_raw(data_dir):
    '''
    Purpose: preprocess oxford metrics derived from profile pictures (part 1)
    Input
        input_dir {string} : path to input_directory (ex, "~/Train")
    Output:
        image_data {pandas DataFrame of float}: unscaled oxford image data
    '''
    # Load data of oxford features extracted from profile picture (face metrics)
    # 7915 entries; some users have no face, some have multiple faces on image.
    # userids with 1+ face on image: 7174 out of 9500 (train set)
    # duplicated entries (userids with > 1 face on same image): 741 in train set
    oxford = pd.read_csv(os.path.join(data_dir, "Image", "oxford.csv"), sep = ',')
    oxford = oxford.sort_values(by=['userId'])
    '''
    NOTE: headPose_pitch has NO RANGE, drop that feature
    '''
    oxford.drop(['headPose_pitch'], axis=1, inplace=True)

    return oxford


def get_likes_kept(data_dir, num_features) -> List[str]:
    '''
    Purpose: get list of likes to keep as features
    Input:
        data_dir {str} : the parent input directory
        num_features {int} : the number of likes to keep as features,
                        starting from those with highest frequencies
    Output:
        freq_like_id {List of strings}: frequency of most frequent likes,
                    (number = num_features), in descending ordered, indexed by like_id
    '''
    #Why return frequency?
    relation = pd.read_csv(os.path.join(data_dir, "Relation", "Relation.csv")) #, index_col=1)
    relation = relation.drop(['Unnamed: 0'], axis=1)
    like_ids_to_keep = relation['like_id'].value_counts(sort=True, ascending=False)[:num_features] #This sorts features by frequency

    #sort like indices (which are the keys associated with the values kepts)
    likes_int64_list = sorted(like_ids_to_keep.keys()) # This sorts indices by like_id
    likes_str_list = [str(l) for l in likes_int64_list]
    return likes_str_list


def get_relations(data_dir: str, sub_ids: List[str], like_ids_to_keep: List[str]):
    '''
    Purpose: preprocess relations dataset ('likes')

    Input:
        data_dir {str} -- the parent input directory
        sub_ids {numpy array of strings} -- the ordered list of userids
        like_ids_to_keep {List[str]} -- The list of page IDs to keep.

    Returns:
        relations_data -- multihot matrix of the like_id. Rows are indexed with userid, entries are boolean.
    '''
    relation = pd.read_csv(os.path.join(data_dir, "Relation", "Relation.csv")) #, index_col=1)
    relation = relation.drop(['Unnamed: 0'], axis=1)

    ## One HUGE step:
    # likes_to_keep = like_ids_to_keep.keys()
    # kept_relations = relation[relation.like_id.isin(likes_to_keep)]
    # multi_hot_relations = pd.get_dummies(kept_relations, columns=["like_id"], prefix="")
    # multi_hot = multi_hot_relations.groupby(("userid")).sum()
    # return multi_hot_relations
    ###
    total_num_pages = len(like_ids_to_keep)
    # Create a multihot likes matrix of booleans (rows = userids, cols = likes), by batch
    batch_size = 1000

    # Create empty DataFrame with sub_ids as index list
    relation_data = pd.DataFrame(sub_ids, columns = ['userid'])
    relation_data.set_index('userid', inplace=True)

    for start_index in range(0, total_num_pages, batch_size):
        end_index = min(start_index + batch_size, total_num_pages)

        # sets are better for membership testing than lists.
        like_ids_for_this_batch = set(like_ids_to_keep[start_index:end_index])

        filtered_table = relation[relation['like_id'].isin(like_ids_for_this_batch)]
        ## THIS is the slow part:
        relHot = pd.get_dummies(filtered_table, columns=['like_id'], prefix="", prefix_sep="")
        ##
        relHot = relHot.groupby(['userid']).sum().astype(float) # this makes userid the index

        relation_data = pd.concat([relation_data, relHot], axis=1, sort=True)

    relation_data = relation_data.reindex(like_ids_to_keep, axis=1)
    relation_data.fillna(0.0, inplace=True)
    relation_data = relation_data.astype("bool")

    # will be different if users in relation.csv are not in sub_ids
    if not np.array_equal(relation_data.index, sub_ids):
        raise Exception(f"""userIds do not match between relation file and id list:
    {relation_data.index}
    {sub_ids}

    """)

    return relation_data


def make_label_dict(labels):
    '''
    Purpose: make dictionnary of labels from pandas DataFrame
    Input:
        labels {pandas DataFrame}: labels ordered per userids (alphabetical order)
    Output:
        labels_dict {dictionary of pandas DataFrames}: labels (one entry per metric) ordered alphabetically
                by userid for the training set, with userids as index.

    '''
    gender = labels['gender']

    age_grps = labels[['age_xx_24', 'age_25_34', 'age_35_49', 'age_50_xx']]

    '''
    Note: : each DataFrames (value) is indexed by userid in labels_dict
    '''
    labels_dict = {}
    labels_dict['userid'] = labels.index
    labels_dict['gender'] = gender
    labels_dict['age_grps'] = age_grps
    labels_dict['ope'] = labels['ope']
    labels_dict['con'] = labels['con']
    labels_dict['ext'] = labels['ext']
    labels_dict['agr'] = labels['agr']
    labels_dict['neu'] = labels['neu']

    return labels_dict


def preprocess_labels(data_dir, sub_ids):
    '''
    Purpose: preprocess entry labels from training set
    Input:
        datadir {string} : path to training data directory
        sub_ids {numpy array of strings}: list of subject ids ordered alphabetically
    Output:
        labels {pandas DataFrame}: labels ordered by userid (alphabetically)
                for the training set, with userids as index.

    '''
    labels = pd.read_csv(os.path.join(data_dir, "Profile", "Profile.csv"))
    labels = labels.sort_values(by=['userid'])
    # check if same subject ids in labels and sub_ids
    if not np.array_equal(labels['userid'].to_numpy(), sub_ids):
        raise Exception('userIds do not match between profiles labels and id list')

    def age_group_id(age_str: str) -> int:
        """Returns the age group category ID (an integer from 0 to 3) for the given age (string)

        Arguments:
            age_str {str} -- the age

        Returns:
            int -- the ID of the age group: 0 for xx-24, 1 for 25-34, 2 for 35-49 and 3 for 50-xx.
        """
        age = int(age_str)
        if age <= 24:
            return 0
        elif age <= 34:
            return 1
        elif age <= 49:
            return 2
        else:
            return 3

    labels = labels.assign(age_group = lambda dt: pd.Series([age_group_id(age_str) for age_str in dt["age"]]))
    # labels = labels.assign(age_xx_24 = lambda dt: pd.Series([int(age) <= 24 for age in dt["age"]]))
    # labels = labels.assign(age_25_34 = lambda dt: pd.Series([25 <= int(age) <= 34 for age in dt["age"]]))
    # labels = labels.assign(age_35_49 = lambda dt: pd.Series([35 <= int(age) <= 49 for age in dt["age"]]))
    # labels = labels.assign(age_50_xx = lambda dt: pd.Series([50 <= int(age) for age in dt["age"]]))

    labels = labels.drop(['Unnamed: 0'], axis=1)
    labels.set_index('userid', inplace=True)

    return labels


def preprocess_train(data_dir, num_likes=10_000, scaling=True):
    '''
    Purpose: preprocesses training dataset (with labels) and returns scaled features,
    labels and parameters to scale the test data set
    Input
        data_dir {string}: path to ~/Train data directory
        num_likes {int}: number of like_ids to keep as features
        scaling {boolean}: if True, Robust scaling applied to data; no scaling if False                
    Output:
        train_features {pandas DataFrame}: vectorized features scaled between 0 and 1
                for each user id in the training set, concatenated for all modalities
                (order = text + image + relation), with userid as DataFrame index.
        **(updated:)features_q10_q90 {tupple of 2 pandas Series}: series of 10th and 90th quantile values of
                text + image features from train dataset, to be used to scale test data.
                Note that the multihot relation features do not necessitate scaling.
        image_means {list of float}: means from oxford dataset to replace missing entries in oxford test set
        likes_kept {list of strings}: ordered likes_ids to serve as columns for test set relation features matrix
        train_labels {pandas DataFrame}: labels ordered by userid (alphabetically)
                for the training set, with userids as index.

    TO CONSIDER: convert outputted pandas to tensorflow tf.data.Dataset...
    https://www.tensorflow.org/guide/data
    '''
    # sub_ids: a numpy array of subject ids ordered alphabetically.
    # text_data: a pandas DataFrame of unscaled text data (liwc and nrc)
    sub_ids, text_data = get_text_data(data_dir)
    # image_data: pandas dataframe of oxford data
    # image_min_max: a tupple of 2 pandas series, the min and max values from oxford training features
    image_data_raw = get_image_raw(data_dir)
    image_means = image_data_raw.iloc[:, 2:].mean().tolist()
    image_data = get_image_clean(sub_ids, image_data_raw, image_means)

    '''
    Note: Scale the text and image data BEFORE concatenating with relations
    Update: scaling w RobustScaler rather than MinMaxScaler algo, due to outliers
    '''
    features_to_scale = pd.concat([text_data, image_data.iloc[:, :-2]], axis=1, sort=False)
    #feat_min = features_to_scale.min()
    #feat_max = features_to_scale.max()
    feat_q10 = features_to_scale.quantile(q = 0.10)
    feat_q90 = features_to_scale.quantile(q = 0.90)

    #feat_scaled = (features_to_scale - feat_min) / (feat_max - feat_min)
    #features_min_max = (feat_min, feat_max)
    if scaling:
        feat_scaled = (features_to_scale - feat_q10) / (feat_q90 - feat_q10)
    else:
        feat_scaled = features_to_scale
    features_q10_q90 = (feat_q10, feat_q90)

    #if DEBUG:
    #    likes_kept = [str(v) for v in range(num_likes)]
    #else:
    likes_kept = get_likes_kept(data_dir, num_likes)

    # multi-hot matrix of likes from train data
    likes_data = get_relations(data_dir, sub_ids, likes_kept)

    # concatenate all scaled features into a single DataFrame
    train_features = pd.concat([feat_scaled, image_data.iloc[:, -2:], likes_data], axis=1, sort=False)

    # DataFrame of training set labels
    train_labels = preprocess_labels(data_dir, sub_ids)

    #return train_features, features_min_max, image_means, likes_kept, train_labels
    return train_features, features_q10_q90, image_means, likes_kept, train_labels


#def preprocess_test(data_dir, min_max_train, image_means_train, likes_kept_train):
def preprocess_test(data_dir, q10_q90_train, image_means_train, likes_kept_train, scaling=True):
    '''
    Purpose: preprocesses test dataset (no labels)
    Input:
        datadir {string}: path to Test data directory
        (**updated)q10_q90_train {tupple of two numpy arrays}: 10th and 90th quantile values for
                concatenated text and image features (from train set)
        image_means_train {list of float}: means from oxford training dataset to replace
                missing entries in oxford test set
        likes_kept_train {list of strings}: most frequent likes_ids from train set
                (ordered by frequency) to serve as columns in relation features matrix
        scaling {boolean}: if True, Robust scaling applied to data; no scaling if False       
    Output:
        test_features {pandas DataFrame}: vectorized features of test set
    '''
    # sub_ids: a numpy array of subject ids ordered alphabetically.
    # text_data: a pandas DataFrame of unscaled text data (liwc and nrc)
    sub_ids, text_data = get_text_data(data_dir)

    # image_data: pandas dataframe of oxford data
    # image_min_max: a tupple of 2 pandas series, the min and max values from oxford training features
    image_data_raw = get_image_raw(data_dir)
    image_data = get_image_clean(sub_ids, image_data_raw, image_means_train)

    '''
    Note: Scale the text and image data BEFORE concatenating with relations
    '''
    features_to_scale = pd.concat([text_data, image_data.iloc[:, :-2]], axis=1, sort=False)
    #feat_min = min_max_train[0]
    #feat_max = min_max_train[1]
    feat_q10 = q10_q90_train[0]
    feat_q90 = q10_q90_train[1]

    #feat_scaled = (features_to_scale - feat_min) / (feat_max - feat_min)
    if scaling:
        feat_scaled = (features_to_scale - feat_q10) / (feat_q90 - feat_q10)
    else:
        feat_scaled = features_to_scale
    
    # multi-hot matrix of likes from train data
    likes_data = get_relations(data_dir, sub_ids, likes_kept_train)

    # concatenate all scaled features into a single DataFrame
    test_features = pd.concat([feat_scaled, image_data.iloc[:, -2:], likes_data], axis=1, sort=False)

    return test_features


def get_train_val_sets(features, labels, val_prop):
    '''
    Purpose: Splits training dataset into a train and a validation set of
    ratio determined by val_prop (x = features, y = labels)
    Input
        features {pandas DataFrame}: vectorized features scaled between 0 and 1
                for each user id in the training set, concatenated for all modalities
                (order = text + image + relation), with userid as DataFrame index.
        labels {pandas DataFrame}: labels ordered by userid (alphabetically)
                for the training set, with userids as index.
        val_prop {float between 0 and 1}: proportion of sample in validation set
                    (e.g. 0.2 = 20% validation, 80% training)
    Output:
        x_train, x_val {pandas DataFrames}: vectorized features for train and validation sets
        y_train, y_val {pandas DataFrames}: train and validation set labels

    TO DO: convert outputted pandas to tensorflow tf.data.Dataset?...
    https://www.tensorflow.org/guide/data
    '''
    # NOTE: UNUSED
    from sklearn import model_selection
    x_train, x_val, y_train, y_val = model_selection.train_test_split(
        features, # training features to split
        labels, # training labels to split
        test_size = val_prop, # between 0 and 1, proportion of sample in validation set (e.g., 0.2)
        shuffle= True,
        #stratify = y_data[:1],
        # random_state = 42  # can use to always obtain the same train/validation split
        )

    return x_train, x_val, y_train, y_val


In [4]:
# to preprocess the training dataset:
# 1. set path to Train directory
# 2. call preprocess_train

train_path = '../Train' #modify if working from other directory

train_features, features_q10_q90, image_means, likes_kept, train_labels = preprocess_train(train_path, num_likes=10_000, scaling=False)



/home/mila/teaching/user07/.local/lib/python3.7/site-packages/ipykernel_launcher.py:76: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [5]:
# split training data into training and validation sets

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    train_features, # training features to split
    train_labels, # training labels to split
    test_size = 0.2, # between 0 and 1, proportion of sample in validation set (e.g., 0.2)
    shuffle= True,
    stratify = train_labels['gender']
    # random_state = 42  # can use to always obtain the same train/validation split
    )


In [6]:
x_train.head()
############################ END OF OLD PREPROCESSING EXTRACTION 

,WC,WPS,Sixltr,Dic,Numerals,funct,pronoun,ppron,i,we,...,10150131036435262,10150136203045543,10150138573815473,10150145087245298,10150145163490188,10150145214185538,10150147152190368,10150154095435553,10150157058260374,10150169313485249
userId,,,,,,,,,,,,,,,,,,,,,
0da7b1011ddc89416d6ac7ebbd4786be,100,4.76,19.00,64.00,0.00,35.00,12.00,7.00,4.00,2.00,...,False,False,False,False,False,False,False,False,False,False
f4532d4a5d06a6a5c75c9605af694598,182,11.38,13.19,81.87,3.30,51.65,13.74,9.89,7.14,0.55,...,False,False,False,False,False,False,False,False,False,False
92b488e28cb1ea9f74bebefeaa152b9c,169,15.36,19.53,79.88,1.78,46.15,17.16,10.65,6.51,0.00,...,False,False,False,False,False,False,False,False,False,False
6689d6569984c1d130057441fb4fde1c,175,87.50,20.57,84.57,0.00,54.29,20.00,16.57,10.86,0.57,...,False,False,False,False,False,False,False,False,False,False
01c41fb8d8a33e01d69c23887327aaf7,190,11.88,16.32,87.37,1.05,55.26,11.05,6.84,4.21,1.58,...,False,False,False,False,False,False,False,False,False,False


In [7]:
x_train.columns[91:156]

Index(['faceRectangle_width', 'faceRectangle_height', 'faceRectangle_left',
       'faceRectangle_top', 'pupilLeft_x', 'pupilLeft_y', 'pupilRight_x',
       'pupilRight_y', 'noseTip_x', 'noseTip_y', 'mouthLeft_x', 'mouthLeft_y',
       'mouthRight_x', 'mouthRight_y', 'eyebrowLeftOuter_x',
       'eyebrowLeftOuter_y', 'eyebrowLeftInner_x', 'eyebrowLeftInner_y',
       'eyeLeftOuter_x', 'eyeLeftOuter_y', 'eyeLeftTop_x', 'eyeLeftTop_y',
       'eyeLeftBottom_x', 'eyeLeftBottom_y', 'eyeLeftInner_x',
       'eyeLeftInner_y', 'eyebrowRightInner_x', 'eyebrowRightInner_y',
       'eyebrowRightOuter_x', 'eyebrowRightOuter_y', 'eyeRightInner_x',
       'eyeRightInner_y', 'eyeRightTop_x', 'eyeRightTop_y', 'eyeRightBottom_x',
       'eyeRightBottom_y', 'eyeRightOuter_x', 'eyeRightOuter_y',
       'noseRootLeft_x', 'noseRootLeft_y', 'noseRootRight_x',
       'noseRootRight_y', 'noseLeftAlarTop_x', 'noseLeftAlarTop_y',
       'noseRightAlarTop_x', 'noseRightAlarTop_y', 'noseLeftAlarOutTip_x',
      

In [8]:
# PATH_IMAGES="/home/mila/teaching/user07/Train/Image/oxford.csv"
# oxford = pd.read_csv(PATH_IMAGES)
# oxford = oxford.rename(columns={"userId":"userid"})
# oxford.info()

In [9]:
oxford_train = x_train.iloc[:,91:156]
oxford_val = x_val.iloc[:,91:156]

In [21]:
oxford_train.columns

Index(['faceRectangle_width', 'faceRectangle_height', 'faceRectangle_left',
       'faceRectangle_top', 'pupilLeft_x', 'pupilLeft_y', 'pupilRight_x',
       'pupilRight_y', 'noseTip_x', 'noseTip_y', 'mouthLeft_x', 'mouthLeft_y',
       'mouthRight_x', 'mouthRight_y', 'eyebrowLeftOuter_x',
       'eyebrowLeftOuter_y', 'eyebrowLeftInner_x', 'eyebrowLeftInner_y',
       'eyeLeftOuter_x', 'eyeLeftOuter_y', 'eyeLeftTop_x', 'eyeLeftTop_y',
       'eyeLeftBottom_x', 'eyeLeftBottom_y', 'eyeLeftInner_x',
       'eyeLeftInner_y', 'eyebrowRightInner_x', 'eyebrowRightInner_y',
       'eyebrowRightOuter_x', 'eyebrowRightOuter_y', 'eyeRightInner_x',
       'eyeRightInner_y', 'eyeRightTop_x', 'eyeRightTop_y', 'eyeRightBottom_x',
       'eyeRightBottom_y', 'eyeRightOuter_x', 'eyeRightOuter_y',
       'noseRootLeft_x', 'noseRootLeft_y', 'noseRootRight_x',
       'noseRootRight_y', 'noseLeftAlarTop_x', 'noseLeftAlarTop_y',
       'noseRightAlarTop_x', 'noseRightAlarTop_y', 'noseLeftAlarOutTip_x',
      

In [11]:
oxford_train[['eyebrowRightOuter_x','eyebrowRightInner_x', 'eyebrowRightOuter_y','eyebrowRightInner_y']]

,eyebrowRightOuter_x,eyebrowRightInner_x,eyebrowRightOuter_y,eyebrowRightInner_y
userId,,,,
0da7b1011ddc89416d6ac7ebbd4786be,78.60000,63.200000,24.200000,23.900000
f4532d4a5d06a6a5c75c9605af694598,138.20000,111.700000,59.000000,48.400000
92b488e28cb1ea9f74bebefeaa152b9c,149.50000,123.500000,88.400000,85.900000
6689d6569984c1d130057441fb4fde1c,172.60000,153.000000,59.400000,59.400000
01c41fb8d8a33e01d69c23887327aaf7,137.20000,113.600000,93.200000,93.900000
...,...,...,...,...
4f1cfb30a2b832daafe8c877f5e17986,77.80000,54.100000,51.600000,58.100000
71056931276dd72a5959b950ee5a4171,120.00000,107.000000,60.600000,59.000000
48ec4381852aa7399ceee39bf1034284,129.31957,106.935944,67.210385,66.812129


In [12]:
def distance(x1,y1,x2,y2):
    return np.sqrt(np.square(x1-x2)+np.square(y1-y2))

In [13]:
"""
Engineering interesting features for:
-gender

-age group

Assumption: Oxford features not useful for personality...

Scaling = divide by face rectangle width/height the new feature

New features:
o eyebrow length AVG X
o width nose root X
o length nose root to tip X should be good...
o width altar AVG X tip - top
o width lips = width upper + width bottom X
o width nose = altar to altar X
o vert eye AVG X
o hori eye AVG X
o transversal length center of eye to nose tip! (normalize transversal) NO
o length mouth X
o bottomEye_to_mouth X avg

Pandas tips:
df.assign(temp_f=lambda x: x.temp_c * 9 / 5 + 32) to assign a new column with result of operation
"""


oxford_train = oxford_train.assign(eyebrow_length=lambda x: (distance(x.eyebrowRightInner_x, x.eyebrowRightInner_y,
                                                                        x.eyebrowRightOuter_x,x.eyebrowRightOuter_y)
                                                             + distance(x.eyebrowLeftInner_x, x.eyebrowLeftInner_y,
                                                                        x.eyebrowLeftOuter_x,x.eyebrowLeftOuter_y)
                                                            )/2/x.faceRectangle_width
                                  )

oxford_train = oxford_train.assign(vert_eye=lambda x: (distance(x.eyeRightInner_x, x.eyeRightInner_y,
                                                                        x.eyeRightOuter_x,x.eyeRightOuter_y)
                                                             + distance(x.eyeLeftInner_x, x.eyeLeftInner_y,
                                                                        x.eyeLeftOuter_x,x.eyeLeftOuter_y)
                                                            )/2/x.faceRectangle_width)

oxford_train = oxford_train.assign(hori_eye=lambda x: (distance(x.eyeRightTop_x, x.eyeRightTop_y,
                                                                        x.eyeRightBottom_x,x.eyeRightBottom_y)
                                                             + distance(x.eyeLeftTop_x, x.eyeLeftTop_y,
                                                                        x.eyeLeftBottom_x,x.eyeLeftBottom_y)
                                                            )/2/x.faceRectangle_width)

oxford_train = oxford_train.assign(width_nose_root=lambda x: distance(x.noseRootLeft_x,x.noseRootLeft_y,
                                                                     x.noseRootRight_x,x.noseRootRight_y)/x.faceRectangle_width )

oxford_train = oxford_train.assign(nose_root_to_tip=lambda x: distance(np.abs(x.noseRootLeft_x-x.noseRootRight_x)/2,np.abs(x.noseRootLeft_y-x.noseRootRight_y)/2,
                                                                     x.noseTip_x,x.noseTip_y)/x.faceRectangle_width )

oxford_train = oxford_train.assign(width_nose=lambda x: distance(x.noseRightAlarOutTip_x, x.noseRightAlarOutTip_y,
                                                                        x.noseLeftAlarOutTip_x,x.noseLeftAlarOutTip_y)/x.faceRectangle_width)

oxford_train = oxford_train.assign(length_mouth=lambda x: distance( x.mouthRight_x, x.mouthRight_y, x.mouthLeft_x, x.mouthLeft_y)/x.faceRectangle_width)

oxford_train = oxford_train.assign(bottomEye_to_mouth=lambda x: (distance( x.mouthRight_x, x.mouthRight_y, x.eyeRightBottom_x, x.eyeRightBottom_y)
                                                                 +distance( x.mouthLeft_x, x.mouthLeft_y, x.eyeLeftBottom_x, x.eyeLeftBottom_y))/2/x.faceRectangle_width)

oxford_train = oxford_train.assign(width_lips=lambda x: (distance( x.upperLipTop_x, x.upperLipTop_y, x.upperLipBottom_x, x.upperLipBottom_y)+
                                  distance( x.underLipTop_x, x.underLipTop_y, x.underLipBottom_x, x.underLipBottom_y))/x.faceRectangle_width)

#oxford_train = oxford_train.assign(pupil_to_nose_tip=lambda x: distance( pupilRight_x, pupilRight_y, upperLipBottom_x, upperLipBottom_y)+
#                                  distance( underLipTop_x, underLipTop_y, underLipBottom_x, underLipBottom_y))

In [14]:
oxford_train.head()

,faceRectangle_width,faceRectangle_height,faceRectangle_left,faceRectangle_top,pupilLeft_x,pupilLeft_y,pupilRight_x,pupilRight_y,noseTip_x,noseTip_y,...,multiface,eyebrow_length,vert_eye,hori_eye,width_nose_root,nose_root_to_tip,width_nose,length_mouth,bottomEye_to_mouth,width_lips
userId,,,,,,,,,,,,,,,,,,,,,
0da7b1011ddc89416d6ac7ebbd4786be,43.0,43.0,39.0,15.0,51.7,28.8,71.6,27.4,58.8,37.2,...,0,0.346420,0.129501,0.040370,0.125925,1.562855,0.318639,0.495545,0.400481,0.072347
f4532d4a5d06a6a5c75c9605af694598,87.0,87.0,53.0,38.0,82.5,62.0,122.7,66.6,88.3,77.9,...,0,0.263808,0.159180,0.065284,0.126044,1.302427,0.312663,0.396313,0.421080,0.084717
92b488e28cb1ea9f74bebefeaa152b9c,88.0,88.0,72.0,73.0,93.4,100.9,136.3,97.5,116.3,122.2,...,0,0.305133,0.152062,0.057525,0.128429,1.872479,0.321036,0.523562,0.361221,0.114977
6689d6569984c1d130057441fb4fde1c,54.0,54.0,125.0,54.0,142.3,69.3,163.4,67.2,140.6,81.8,...,1,0.249099,0.116041,0.047728,0.137087,2.952325,0.350078,0.393121,0.477167,0.101014
01c41fb8d8a33e01d69c23887327aaf7,67.0,67.0,76.0,82.0,96.3,101.1,125.5,100.9,108.4,117.0,...,0,0.347459,0.130399,0.031573,0.111950,2.342329,0.341794,0.470341,0.432254,0.081635


In [15]:
oxford_train.columns

Index(['faceRectangle_width', 'faceRectangle_height', 'faceRectangle_left',
       'faceRectangle_top', 'pupilLeft_x', 'pupilLeft_y', 'pupilRight_x',
       'pupilRight_y', 'noseTip_x', 'noseTip_y', 'mouthLeft_x', 'mouthLeft_y',
       'mouthRight_x', 'mouthRight_y', 'eyebrowLeftOuter_x',
       'eyebrowLeftOuter_y', 'eyebrowLeftInner_x', 'eyebrowLeftInner_y',
       'eyeLeftOuter_x', 'eyeLeftOuter_y', 'eyeLeftTop_x', 'eyeLeftTop_y',
       'eyeLeftBottom_x', 'eyeLeftBottom_y', 'eyeLeftInner_x',
       'eyeLeftInner_y', 'eyebrowRightInner_x', 'eyebrowRightInner_y',
       'eyebrowRightOuter_x', 'eyebrowRightOuter_y', 'eyeRightInner_x',
       'eyeRightInner_y', 'eyeRightTop_x', 'eyeRightTop_y', 'eyeRightBottom_x',
       'eyeRightBottom_y', 'eyeRightOuter_x', 'eyeRightOuter_y',
       'noseRootLeft_x', 'noseRootLeft_y', 'noseRootRight_x',
       'noseRootRight_y', 'noseLeftAlarTop_x', 'noseLeftAlarTop_y',
       'noseRightAlarTop_x', 'noseRightAlarTop_y', 'noseLeftAlarOutTip_x',
      

In [16]:
"""
Keep the good columns
"""
oxford_train_modified = oxford_train[['faceRectangle_width', 'facialHair_mustache', 'facialHair_beard', 'facialHair_sideburns', 'eyebrow_length', 'vert_eye', 'hori_eye', 'width_nose_root',
       'nose_root_to_tip', 'width_nose', 'length_mouth', 'bottomEye_to_mouth',
       'width_lips','headPose_yaw', 'noface', 'multiface']]

In [17]:
#it got all the columns right
oxford_train_modified

,faceRectangle_width,facialHair_mustache,facialHair_beard,facialHair_sideburns,eyebrow_length,vert_eye,hori_eye,width_nose_root,nose_root_to_tip,width_nose,length_mouth,bottomEye_to_mouth,width_lips,headPose_yaw,noface,multiface
userId,,,,,,,,,,,,,,,,
0da7b1011ddc89416d6ac7ebbd4786be,43.000000,0.5000,0.400000,0.400000,0.346420,0.129501,0.040370,0.125925,1.562855,0.318639,0.495545,0.400481,0.072347,-9.400000,0,0
f4532d4a5d06a6a5c75c9605af694598,87.000000,0.0000,0.000000,0.000000,0.263808,0.159180,0.065284,0.126044,1.302427,0.312663,0.396313,0.421080,0.084717,-24.100000,0,0
92b488e28cb1ea9f74bebefeaa152b9c,88.000000,0.1000,0.300000,0.100000,0.305133,0.152062,0.057525,0.128429,1.872479,0.321036,0.523562,0.361221,0.114977,-0.200000,0,0
6689d6569984c1d130057441fb4fde1c,54.000000,0.0000,0.200000,0.000000,0.249099,0.116041,0.047728,0.137087,2.952325,0.350078,0.393121,0.477167,0.101014,-41.100000,0,1
01c41fb8d8a33e01d69c23887327aaf7,67.000000,0.4000,0.400000,0.200000,0.347459,0.130399,0.031573,0.111950,2.342329,0.341794,0.470341,0.432254,0.081635,-3.600000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4f1cfb30a2b832daafe8c877f5e17986,71.000000,0.0000,0.000000,0.000000,0.289979,0.151415,0.056390,0.119528,1.400435,0.312044,0.342934,0.477350,0.107828,-27.700000,0,0
71056931276dd72a5959b950ee5a4171,40.000000,0.0000,0.000000,0.000000,0.323438,0.135366,0.058974,0.162788,3.026947,0.306961,0.432796,0.420524,0.071041,-4.200000,0,0
48ec4381852aa7399ceee39bf1034284,78.358307,0.0941,0.107732,0.080884,0.286042,0.144742,0.060173,0.125329,1.712697,0.287348,0.406576,0.422805,0.097812,-0.536096,1,0


In [18]:
oxford_train_modified.columns

Index(['faceRectangle_width', 'facialHair_mustache', 'facialHair_beard',
       'facialHair_sideburns', 'eyebrow_length', 'vert_eye', 'hori_eye',
       'width_nose_root', 'nose_root_to_tip', 'width_nose', 'length_mouth',
       'bottomEye_to_mouth', 'width_lips', 'headPose_yaw', 'noface',
       'multiface'],
      dtype='object')

In [19]:
len(oxford_train_modified.columns)

16